# [3주차] 심화과제 - Pre-trained 모델로 효율적인 NLP 모델 학습하기

#### Q1) 어떤 task를 선택하셨나요?
- MNLI task
MNLI는 두 문장이 주어졌을 때 논리적으로 연결이 되어 있는지, 서로 모순되는지, 아니면 아예 무관한지 분류하는 문제이다.
BERT는 sentence pair classification에 특화되어 있어 MNLI와 같이 두 문장 간의 복잡한 의미 관계를 파악하는데 효과적이다.
BERT와 비슷한 성능을 가지면서 사이즈는 작은 DistilBERT를 사용하였다.

#### Q2) 모델은 어떻게 설계하셨나요? 설계한 모델의 입력과 출력 형태가 어떻게 되나요?
> 모델의 입력과 출력 형태 또는 shape을 정확하게 기술


#### Q3) 실제로 pre-trained 모델을 fine-tuning했을 때 loss curve은 어떻게 그려지나요? 그리고 pre-train 하지 않은 Transformer를 학습했을 때와 어떤 차이가 있나요? 
> 비교 metric은 loss curve, accuracy, 또는 test data에 대한 generalization 성능 등을 활용.
> +)이외에도 기계 번역 같은 문제에서 활용하는 BLEU 등의 metric을 마음껏 활용 가능
- 



In [ ]:
%pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

DistilBERT pre-training 때 사용한 tokenizer를 불러온다.

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# DistilBERT 모델용 tokenizer 로드 (pretrained)
# 이 tokenizer는 문장을 토큰화해서 모델이 이해할 수 있는 input_ids로 변환해줌
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Kaggle 데이터셋을 다운로드 받는다.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/unlocking-language-understanding-with-the-multin")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
import os

# 1. 데이터 로드
def load_mnli_data(path):
    train_path = os.path.join(path, 'multinli_1.0_train.txt')
    val_path = os.path.join(path, 'multinli_1.0_dev_matched.txt')
    
    # tab으로 구분된 텍스트 파일 읽기
    train_df = pd.read_csv(train_path, sep='\t')
    val_df = pd.read_csv(val_path, sep='\t')
    
    # 필요한 컬럼만 선택
    columns = ['sentence1', 'sentence2', 'gold_label']
    train_df = train_df[columns]
    val_df = val_df[columns]
    
    return train_df, val_df

# 2. MNLI 데이터셋 클래스 정의
class MNLIDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length
        
        # 레이블 매핑
        self.label_map = {
            'entailment': 0,
            'contradiction': 1,
            'neutral': 2
        }
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # 텍스트 인코딩
        encoding = self.tokenizer(
            row['sentence1'],
            row['sentence2'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # 배치 차원 제거
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        
        # 레이블 변환
        label = self.label_map.get(row['gold_label'], -1)  # -1은 invalid 레이블용
        encoding['labels'] = torch.tensor(label)
        
        return encoding

# 3. 데이터 준비 및 데이터로더 설정
def prepare_dataloaders(path, batch_size=16):
    # 토크나이저 초기화
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    
    # 데이터 로드
    train_df, val_df = load_mnli_data(path)
    
    # 유효하지 않은 레이블 제거
    train_df = train_df[train_df['gold_label'].isin(['entailment', 'contradiction', 'neutral'])]
    val_df = val_df[val_df['gold_label'].isin(['entailment', 'contradiction', 'neutral'])]
    
    # 데이터셋 생성
    train_dataset = MNLIDataset(train_df, tokenizer)
    val_dataset = MNLIDataset(val_df, tokenizer)
    
    # 데이터로더 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )
    
    return train_loader, val_loader

# 4. 사용 예시
if __name__ == "__main__":
    # kagglehub에서 다운로드 받은 경로 사용
    path = kagglehub.dataset_download("thedevastator/unlocking-language-understanding-with-the-multin")
    
    # 데이터로더 생성
    train_loader, val_loader = prepare_dataloaders(path)
    
    # 데이터 형태 확인
    for batch in train_loader:
        print("입력 데이터 형태:")
        print(f"Input IDs shape: {batch['input_ids'].shape}")
        print(f"Attention mask shape: {batch['attention_mask'].shape}")
        print(f"Labels shape: {batch['labels'].shape}")
        break